In [15]:
import os
import glob
import torch
from PIL import Image
from torchvision import transforms
from torchvision.utils import save_image
from fire import Fire
from models.vae3d import IV_VAE  # or your 2D VAE

In [16]:
image_path = "./image/input.png"
save_path = "./image/output.png"

height = 1024    
width = 1024 
z_dim = 8
dim = 64

In [17]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

vae3d = IV_VAE(z_dim, dim).to(device=device, dtype=torch.bfloat16)
vae3d.requires_grad_(False)
vae3d.eval()

total_param = sum(p.numel() for p in vae3d.parameters())
print("total params: ", total_param)

pretrained_model: ivvae_z8_dim64
total params:  107379150


In [18]:
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.CenterCrop((height, width)),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x * 2 - 1)
    ])
    img_tensor = transform(image).unsqueeze(0).to(device=device, dtype=torch.bfloat16)  # [1, 3, H, W]
    img_tensor = img_tensor.unsqueeze(2)  # [1, 3, 1, H, W]
    return img_tensor
    


In [19]:
datasets =['clic', 'celeb', 'metfaces']
data = {}
for dataset in datasets:
    data_dir = os.path.join("/cs/student/projects1/2023/rcheng/data", dataset)
    images = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.png') or f.endswith('.jpg')]
    if len(images) > 1000:
        images = images[:1000]  # Limit to first 1000 images for testing
    data[dataset] = images
    
time, encode_t, decode_t = 0, 0, 0 

In [20]:
from tqdm.notebook import tqdm
save_path = "./results/"
def process_images(images, dataset):
    for image in tqdm(images):
        image_tensor = load_image(image)  # expected shape: [1, 3, 1, H, W]

        start_total = torch.cuda.Event(enable_timing=True)
        end_total = torch.cuda.Event(enable_timing=True)
        start_encode = torch.cuda.Event(enable_timing=True)
        end_encode = torch.cuda.Event(enable_timing=True)
        start_decode = torch.cuda.Event(enable_timing=True)
        end_decode = torch.cuda.Event(enable_timing=True)

        start_total.record()
        print(f"Processing {image}...")
        with torch.no_grad():
            start_encode.record()
            latent = vae3d.encode(image_tensor)
            end_encode.record()

            start_decode.record()
            output = vae3d.decode(latent)
            end_decode.record()

        end_total.record()
        torch.cuda.synchronize()
        
        
        global time, encode_t, decode_t
        time += start_total.elapsed_time(end_total)
        encode_t += start_encode.elapsed_time(end_encode)
        decode_t += start_decode.elapsed_time(end_decode)   

        print(f"Saving result of {os.path.basename(image)}...")

        # Save input image
        input_img = image_tensor.squeeze(0).squeeze(1)  # [3, H, W]
        input_img = torch.clamp((input_img + 1) / 2, 0, 1)
        save_input_path = os.path.join(save_path, dataset, "input", os.path.basename(image))
        os.makedirs(os.path.dirname(save_input_path), exist_ok=True)
        save_image(input_img.to(dtype=torch.float32), save_input_path)

        # Save output image
        output_img = output.squeeze(0).squeeze(1)  # [3, H, W]
        output_img = torch.clamp((output_img + 1) / 2, 0, 1)
        save_output_path = os.path.join(save_path, dataset, "output", os.path.basename(image))
        os.makedirs(os.path.dirname(save_output_path), exist_ok=True)
        save_image(output_img.to(dtype=torch.float32), save_output_path)

In [ ]:

for dataset, images in tqdm(data.items()):
    process_images(images, dataset)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

Processing /cs/student/projects1/2023/rcheng/data/clic/himanshu-singh-gurjar-iSi02D_Qx_w-unsplash.png...
Saving result of himanshu-singh-gurjar-iSi02D_Qx_w-unsplash.png...
Processing /cs/student/projects1/2023/rcheng/data/clic/randy-tarampi-a5KMylND9Bo-unsplash.png...
Saving result of randy-tarampi-a5KMylND9Bo-unsplash.png...
Processing /cs/student/projects1/2023/rcheng/data/clic/aaron-greenwood-x9zy_1oHjqw-unsplash.png...
Saving result of aaron-greenwood-x9zy_1oHjqw-unsplash.png...
Processing /cs/student/projects1/2023/rcheng/data/clic/oscar-keys-iWNFqEYpmmo-unsplash.png...
Saving result of oscar-keys-iWNFqEYpmmo-unsplash.png...
Processing /cs/student/projects1/2023/rcheng/data/clic/lynda-hinton-HayTQnEEtrQ-unsplash.png...
Saving result of lynda-hinton-HayTQnEEtrQ-unsplash.png...
Processing /cs/student/projects1/2023/rcheng/data/clic/jezael-melgoza-7H77FWkK_x4-unsplash.png...
Saving result of jezael-melgoza-7H77FWkK_x4-unsplash.png...
Processing /cs/student/projects1/2023/rcheng/data/

  0%|          | 0/1000 [00:00<?, ?it/s]

Processing /cs/student/projects1/2023/rcheng/data/celeb/20015.jpg...
Saving result of 20015.jpg...
Processing /cs/student/projects1/2023/rcheng/data/celeb/16981.jpg...
Saving result of 16981.jpg...
Processing /cs/student/projects1/2023/rcheng/data/celeb/23584.jpg...
Saving result of 23584.jpg...
Processing /cs/student/projects1/2023/rcheng/data/celeb/20698.jpg...
Saving result of 20698.jpg...
Processing /cs/student/projects1/2023/rcheng/data/celeb/08095.jpg...
Saving result of 08095.jpg...
Processing /cs/student/projects1/2023/rcheng/data/celeb/23638.jpg...
Saving result of 23638.jpg...
Processing /cs/student/projects1/2023/rcheng/data/celeb/17073.jpg...
Saving result of 17073.jpg...
Processing /cs/student/projects1/2023/rcheng/data/celeb/01553.jpg...
Saving result of 01553.jpg...
Processing /cs/student/projects1/2023/rcheng/data/celeb/29887.jpg...
Saving result of 29887.jpg...
Processing /cs/student/projects1/2023/rcheng/data/celeb/10758.jpg...
Saving result of 10758.jpg...
Processing

  0%|          | 0/495 [00:00<?, ?it/s]

Processing /cs/student/projects1/2023/rcheng/data/metfaces/436683-00.png...
Saving result of 436683-00.png...
Processing /cs/student/projects1/2023/rcheng/data/metfaces/436688-00.png...
Saving result of 436688-00.png...
Processing /cs/student/projects1/2023/rcheng/data/metfaces/436684-00.png...
Saving result of 436684-00.png...
Processing /cs/student/projects1/2023/rcheng/data/metfaces/436798-00.png...
Saving result of 436798-00.png...
Processing /cs/student/projects1/2023/rcheng/data/metfaces/436911-00.png...
Saving result of 436911-00.png...
Processing /cs/student/projects1/2023/rcheng/data/metfaces/436871-00.png...
Saving result of 436871-00.png...
Processing /cs/student/projects1/2023/rcheng/data/metfaces/436928-00.png...
Saving result of 436928-00.png...
Processing /cs/student/projects1/2023/rcheng/data/metfaces/436941-00.png...
Saving result of 436941-00.png...
Processing /cs/student/projects1/2023/rcheng/data/metfaces/436928-01.png...
Saving result of 436928-01.png...
Processing

In [25]:

total_item = sum(len(images) for images in data.values())
total_time = time / total_item 
encode_time = encode_t / total_item
decode_time = decode_t / total_item 

from myutils.mail_notifyer import send_email_mailjet 
send_email_mailjet("cn.aiden.cheng@gmail.com")


✅ Mailjet 邮件发送成功


In [26]:
print(f"Average total time per image: {total_time:.2f} ms")
print(f"Average encoding time per image: {encode_time:.2f} ms")
print(f"Average decoding time per image: {decode_time:.2f} ms")

Average total time per image: 3102.52 ms
Average encoding time per image: 1046.06 ms
Average decoding time per image: 2056.26 ms
